In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
# 下载MNIST数据
"""
import os
import gzip
import shutil
import urllib

def download_file(url, dest, expected_byte=None, unzip_and_remove=False):
    if os.path.exists(dest) or os.path.exists(dest[:-3]):
        print('%s already exists' %dest)
    else:
        print('Downloading %s' %url)
        local_file, _ = urllib.request.urlretrieve(url, dest)
        file_stat = os.stat(dest)
        if expected_byte:
            if file_state.st_size == expected_byte:
                print('Successfully download %s' %dest)
        
        if unzip_and_remove:
            with gzip.open(dest, 'rb') as f_in, open(dest[:-3],'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
            os.remove(dest)

os.makedirs('./data/mnist')
download_url = 'http://yann.lecun.com/exdb/mnist'
filenames = ['train-images-idx3-ubyte.gz',
              'train-labels-idx1-ubyte.gz',
              't10k-images-idx3-ubyte.gz',
              't10k-labels-idx1-ubyte.gz']
expected_bytes = [9912422, 28881, 1648877, 4542]
for filename, byte in zip(filenames, expected_bytes):
    download_url = os.path.join(download_url, filename)
    local_dest = os.path.join('./data/mnist', filename)
    download_file(download_url, local_dest, byte, True)
"""

In [2]:
# 读取数据

import os
import struct

def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test

In [3]:
# 建模

learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

train, val, test = read_mnist('./data/mnist', flatten=True)
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # 随机打乱
train_data = train_data.batch(batch_size) # 小批量

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes) # 迭代器
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data) # 初始化迭代器
test_init = iterator.make_initializer(test_data)

w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01)) # 随机初始化
b = tf.get_variable(name='bais', shape=(1, 10), initializer=tf.zeros_initializer())
logits = tf.matmul(img, w) + b
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels = label, name='entropy') # 交叉熵
loss = tf.reduce_mean(entropy, name='loss') # 平均

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss) # Adam梯度下降

preds = tf.nn.softmax(logits) # 预测
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1)) # 比较索引
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# 训练

writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # variables初始化
    
    for i in range(n_epochs):
        sess.run(train_init) # 训练集初始化
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    
    sess.run(test_init) # 初始化测试集
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    
    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.36437368930079217
Average loss epoch 1: 0.29693095027360805
Average loss epoch 2: 0.286031210890343
Average loss epoch 3: 0.27685066601911257
Average loss epoch 4: 0.27465352317275005
Average loss epoch 5: 0.2732113846512728
Average loss epoch 6: 0.26956073867027147
Average loss epoch 7: 0.2677966321970141
Average loss epoch 8: 0.2661396531691385
Average loss epoch 9: 0.2632947860069053
Average loss epoch 10: 0.26395187223719996
Average loss epoch 11: 0.261479722379252
Average loss epoch 12: 0.2603518205153388
Average loss epoch 13: 0.26035910487868064
Average loss epoch 14: 0.2595602630182754
Average loss epoch 15: 0.26246113272946936
Average loss epoch 16: 0.25773913480861244
Average loss epoch 17: 0.2547619345229726
Average loss epoch 18: 0.2591352727697339
Average loss epoch 19: 0.25511718898318536
Average loss epoch 20: 0.25691213228328286
Average loss epoch 21: 0.2549752280116081
Average loss epoch 22: 0.25502378075968385
Average loss epoch 23: 0.256563073